In [1]:
import pandas as pd
import os
import re
import pysyge

In [2]:
for files in os.listdir():
    temp = files.split('.')
    try:
        if temp[1] == 'txt':
            file = files
    except:
        continue

In [3]:
text = open(file, 'r')
log = text.read()

In [78]:
text.close()

In [79]:
spllog=log.split()

In [80]:
count = 0
for i in spllog:
    if  i == 'shop_api':
        spllog.pop(count)
        spllog.pop(count)
    elif i == 'INFO:':
        spllog.pop(count)
    count = count+1

In [81]:
csv_text = 'Date;Time;Code;IP;Https\n'
count = 0
for i in spllog:
    if count < 4:
        csv_text=csv_text+i+';' 
        count+=1
    elif count == 4:
        csv_text=csv_text+i+'\n'  
        count=0

KeyboardInterrupt: 

In [ ]:
text = open('logs_to_csv.csv', 'w')
log = text.write(csv_text)

In [ ]:
text.close()

======= Location ======

In [7]:
df = pd.read_csv('logs_to_csv.csv', sep=';')

In [8]:
ip = set(df.IP)

In [9]:
geodata = pysyge.GeoLocator('SxGeoCity.dat', pysyge.pysyge.MODE_BATCH  | pysyge.pysyge.MODE_MEMORY)

In [10]:
locations = geodata.get_locations(ip, detailed=True)

In [16]:
loc = []
for i in locations:
    if i:
        loc.append(str(i['info']['country']['name_en']))
    else:
        loc.append(str(i))

In [21]:
list_loc=[]
for i in loc:
    if i == 'True' or i == 'False':
        list_loc.append('Not found')
    else:
        list_loc.append(i)

In [22]:
list_ip = []
for i in ip:
        list_ip.append(i)

In [23]:
df_Loction = pd.DataFrame()
df_Loction['IP']=list_ip
df_Loction['Country'] = list_loc

In [24]:
df_Loction.to_csv('Location.csv', sep = ';',index=False, encoding='Windows 1251')

In [ ]:
#print('%s (%s) calling. All the circuits are busy.' % (location['info']['city']['name_en'], location['info']['country']['iso']))

===== Категории =====

In [87]:
df = pd.read_csv('logs_to_csv.csv', sep=';')

In [123]:
list =[]
for i in df.Https: 
    if len(re.findall(r'user_id*',i))==0 and len(re.findall(r'success_pa*',i))==0 and len(re.findall(r'goods_id*',i))==0:
        spl = re.findall(r'all_to_the_bottom.*',i)[0].split('/')
        if len(spl)>2:
            section = spl[1]
        else:
            section = spl[0]
        list.append(section)

In [126]:
category = []
for i in set(list):
    category.append(i)

In [128]:
df_Category = pd.DataFrame({'Category':category})

In [129]:
df_Category['ID_Category']=df_Category.index

In [130]:
df_Category.to_csv('Category.csv',sep=';', index=False)

======= Товары ======

In [132]:
df = pd.read_csv('logs_to_csv.csv', sep=';')

In [133]:
dfg = pd.read_csv('Location.csv',sep=';')

In [134]:
dict = {}
for j in dfg.IP:
    https = df[df['IP']==j]['Https'] 
    s = ''
    for i in https:
        if len(re.findall(r'goods_id*',i))!=0: 
            
            spl = re.findall(r'goods_id.*',i)[0].split('&')
            goods= re.findall(r'\d.*',spl[0])[0]

            product = re.findall(r'all_to_the_bottom.com.*',s)[0].split('/')
            
            dict[goods]=product[len(product)-3]+'/'+product[len(product)-2]

        s = i 

In [135]:
ID_product = []
Goods = []
Category = []
for i in dict.keys():
    ID_product.append(i)
    spl = re.split('/',dict[i])
    Category.append(spl[0])
    Goods.append(spl[1])

In [136]:
df = pd.DataFrame({'Product':Goods,
                  'Category':Category,
                  'ID_Product':ID_product})

In [137]:
df_Category = pd.read_csv('Category.csv',sep=';')

In [138]:
df_Product = pd.merge(df_Category,df,on ='Category')

In [139]:
df_Product = df_Product.drop('Category', 1) 

In [140]:
df_Product = df_Product.rename(columns={'ID_Category': 'Category'}) 

In [144]:
df_Product.to_csv('Product.csv',sep=';', index=False)

======= Изменения формата Date=========

In [ ]:
df = pd.read_csv('logs_to_csv.csv', sep=';')
list = []
for i in df.Date:
    list.append(i.replace('-','.'))
df.Date=list
df.to_csv('logs_to_csv.csv',sep=';',index=False)

============== Корзина ==============

In [145]:
df = pd.read_csv('logs_to_csv.csv', sep=';')

In [146]:
Date=[]
Time=[]
IP = []
Goods=[]
Amount=[]
Cart_id=[]
for index, row in df.iterrows():
    if len(re.findall(r'goods_id*',row.Https))!=0:
        spl = re.findall(r'goods_id.*',row.Https)[0].split('&')
        goods = re.findall(r'\d.*',spl[0])[0]
        amount = re.findall(r'\d.*',spl[1])[0]
        cart_id = re.findall(r'\d.*',spl[2])[0]
        Date.append(row['Date'])
        Time.append(row['Time'])
        IP.append(row['IP'])
        Goods.append(goods)
        Amount.append(amount)
        Cart_id.append(cart_id)

In [147]:
dfg = pd.DataFrame({'Date':Date,'Time':Time,'IP':IP,'Goods':Goods,'Amount':Amount,'Cart_id':Cart_id})

In [ ]:
dfg['ID_Basket'] = dfg.index

In [148]:
dfg.to_csv('Basket.csv',sep=';',index=False)

============ Покупка ===========



In [149]:
df = pd.read_csv('logs_to_csv.csv', sep=';')

In [150]:
Date=[]
Time=[]
IP = []
Cart_id=[]
User_id=[]
Success_pay=[]

for index, row in df.iterrows():
    if len(re.findall(r'user_id*',row.Https))!=0:
        spl = re.findall(r'user_id.*',row.Https)[0].split('&')
        user_id = re.findall(r'\d.*',spl[0])[0]
        cart_id = re.findall(r'\d.*',spl[1])[0]
        
        Date.append(row['Date'])
        Time.append(row['Time'])
        IP.append(row['IP'])
        User_id.append(user_id)
        Cart_id.append(cart_id)

              
    if len(re.findall(r'success_pay*',row.Https))!=0:
        success_pay = re.findall(r'\d*\d',re.findall(r'success_pay.*',row.Https)[0])
        Success_pay.append(success_pay[0])

In [151]:
dfg1 = pd.DataFrame({'Date':Date,'Time':Time,'IP':IP,'User_id':User_id,'Cart_id':Cart_id})
dfg2 = pd.DataFrame({'Cart_id':Success_pay,'Success_pay':Success_pay})

In [152]:
dfg = pd.merge(dfg1,dfg2, on = 'Cart_id',how='left')

In [153]:
list = []
for i in dfg.Success_pay:
    if len(i)>0:
        list.append(True)
    else:
        list.append(False)
dfg.Success_pay = list

In [154]:
dfg.to_csv('Buy.csv',sep=';',index=False)

========== Посещения =========

In [182]:
df = pd.read_csv('logs_to_csv.csv', sep=';')

In [183]:
Date=[]
Time=[]
IP = []
Section=[]
Product=[]

for index, row in df.iterrows():
    
    if len(re.findall(r'user_id*',row.Https))==0 and len(re.findall(r'success_pa*',row.Https))==0 and len(re.findall(r'goods_id*',row.Https))==0:
        spl = re.findall(r'all_to_the_bottom.*',row.Https)[0].split('/')
        if len(spl)==4:   
            section = spl[1]
            product = spl[2]
        elif len(spl)==3:
            section = spl[1]
            product =  None
        else:
            section = spl[0]
            product =  None
            
        Date.append(row['Date'])
        Time.append(row['Time'])
        IP.append(row['IP'])
        Section.append(section)
        Product.append(product)

In [184]:
dfg = pd.DataFrame({'Date':Date,'Time':Time,'IP':IP,'Category':Section,'Product':Product})

In [187]:
df_Visit = pd.merge(dfg,df_Category, on = 'Category')

In [188]:
df_Visit.Category = df_Visit.ID_Category

In [189]:
df_Visit = df_Visit.drop('ID_Category', 1) 

In [190]:
df = pd.DataFrame()
df['ID_Product'] = df_Product.ID_Product
df['Product'] = df_Product.Product

In [191]:
df_Visit = pd.merge(df_Visit,df, on = 'Product', how ='left')

In [193]:
df_Visit.Product = df_Visit.ID_Product
df_Visit = df_Visit.drop('ID_Product', 1) 

In [194]:
df_Visit['ID_Visit'] = df_Visit.index

In [ ]:
list = []
for i in df_Visit.Time:
    spl = re.split(':',i) 
    if int(spl[0]) < 4 :
        list.append('Night')
    elif int(spl[0])>3 and int(spl[0])<12 :
        list.append('Morning')
    elif int(spl[0])>11 and int(spl[0])<17 :
        list.append('Day')
    else: 
        list.append('Evening')
        
df_Visit['Times_of_Day'] = list

In [12]:
df_Visit.to_csv('Visit.csv', sep=';',index=False)

In [13]:
df = pd.read_csv('Buy.csv', sep=';')

In [15]:
len(set(df.IP))

655

In [16]:
len(set(df.User_id))

655

In [198]:
len(df[df['Country']=='False'])

152